### Imports

In [0]:

# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib



%md
## Analysis of S3 Bucket Mounting

In this section, we mount an S3 bucket onto Databricks using the provided function `mount_s3_bucket()`. This function reads AWS keys from a Delta table and uses them to mount the S3 bucket. Below is the implementation of the function:


In [0]:


def mount_s3_bucket(delta_table_path):
    """
    Mounts an S3 bucket onto Databricks.

    Parameters:
    delta_table_path (str): The path to the Delta table containing AWS keys.

    Returns:
    None
    """
    # Load AWS keys from Delta table
    aws_keys_df = spark.read.format("delta").load(delta_table_path)

    # Extract AWS access key and secret key
    access_key = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
    secret_key = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']

    # Encode the secret key
    encoded_secret_key = urllib.parse.quote(string=secret_key, safe="")

    # AWS S3 bucket name
    aws_s3_bucket = "s3_bucket"
    # Mount name for the bucket
    mount_name = "<desired_mount_path>"
    # Source URL
    source_url = "s3n://{0}:{1}@{2}".format(access_key, encoded_secret_key, aws_s3_bucket)
    # Mount the drive
    dbutils.fs.mount(source_url, mount_name)

# Call the function with the Delta table path
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
mount_s3_bucket(delta_table_path)
